In [ ]:
# Setup directories
from pathlib import Path

basedir = Path().absolute()
libdir = basedir.parent.parent.parent

# Run with local updates
import sys

sys.path.append(str(libdir))

In [ ]:
# Setup logging
import logging

# Notebook logging
f = logging.Formatter("[%(asctime)s] %(levelname)s - %(message)s")
sh = logging.StreamHandler(sys.stdout)
sh.setFormatter(f)
L = logging.getLogger()
L.handlers = [sh]
L.setLevel(logging.INFO)

# Specific libraries with different logging levels
qcl = logging.getLogger("ioos_qc")
qcl.handlers = [sh]
qcl.setLevel(logging.DEBUG)
qcl.propagate = False

In [3]:
import json
import os
import shutil
import tempfile
from datetime import datetime, timedelta

import netCDF4 as nc4
import numpy as np
import pandas as pd
import xarray as xr
from bokeh import plotting
from cftime import date2num, num2date
from erddapy import ERDDAP
from ioos_qc.config import Config
from ioos_qc.config_creator import (
    QC_CONFIG_CREATOR_SCHEMA,
    CreatorConfig,
    QcConfigCreator,
    QcVariableConfig,
)
from ioos_qc.plotting import (
    bokeh_multi_plot,
    bokeh_plot_collected_result,
    bokeh_plot_collected_results,
)
from ioos_qc.results import collect_results
from ioos_qc.stores import CFNetCDFStore, NetcdfStore
from ioos_qc.streams import NetcdfStream, XarrayStream

# TODO: fix aggregate method

In [4]:
# Local specific variables
qc_output_path = "./output/storms_qc/data_output"
os.makedirs(qc_output_path, exist_ok=True)

climatology_path = "./output/storms_qc/ioos_qc/resources"
os.makedirs(climatology_path, exist_ok=True)

In [5]:
# set up ERDDAP
erddap_server = "https://ferret.pmel.noaa.gov/pmel/erddap"

dataset_id = "sd1054"
# sd1054 bounds
query_min_t = "2021-01-04T20:00:00Z"
query_max_t = "2021-03-15T16:45:00Z"
query_min_y = 44.7192672
query_max_y = 47.2086944
query_min_x = -131.1743104
query_max_x = -126.6228352

e = ERDDAP(server=erddap_server, protocol="tabledap")
e.response = "ncCF"
e.dataset_id = dataset_id
e.constraints = {
    "time>=": query_min_t,
    "time<=": query_max_t,
    "latitude>=": query_min_y,
    "latitude<=": query_max_y,
    "longitude>=": query_min_x,
    "longitude<=": query_max_x,
}

data_xr_full = e.to_xarray()
# data_nc = e.to_ncCF()

n_obs = data_xr_full.dims["obs"]
data_xr = data_xr_full.sel(obs=range(0, n_obs))
data_xr

<xarray.Dataset>
Dimensions:                         (obs: 1064, trajectory: 1)
Coordinates:
  * trajectory                      (trajectory) object '1054.0'
    latitude                        (obs) float64 ...
    longitude                       (obs) float64 ...
    time                            (obs) datetime64[ns] ...
Dimensions without coordinates: obs
Data variables:
    rowSize                         (trajectory) int32 ...
    SOG                             (obs) float64 ...
    COG                             (obs) float64 ...
    HDG                             (obs) float64 ...
    HDG_WING                        (obs) float64 ...
    WING_ANGLE                      (obs) float64 ...
    UWND_MEAN                       (obs) float64 ...
    UWND_STDDEV                     (obs) float64 ...
    VWND_MEAN                       (obs) float64 ...
    VWND_STDDEV                     (obs) float64 ...
    WWND_MEAN                       (obs) float64 ...
    WWND_STDDEV                     (obs) float64 ...
    GUST_WND_MEAN                   (obs) float64 ...
    GUST_WND_STDDEV                 (obs) float64 ...
    WIND_MEASUREMENT_HEIGHT_MEAN    (obs) float64 ...
    WIND_MEASUREMENT_HEIGHT_STDDEV  (obs) float64 ...
    TEMP_AIR_MEAN                   (obs) float64 ...
    TEMP_AIR_STDDEV                 (obs) float64 ...
    RH_MEAN                         (obs) float64 ...
    RH_STDDEV                       (obs) float64 ...
    BARO_PRES_MEAN                  (obs) float64 ...
    BARO_PRES_STDDEV                (obs) float64 ...
    PAR_AIR_MEAN                    (obs) float64 ...
    PAR_AIR_STDDEV                  (obs) float64 ...
    TEMP_IR_SEA_WING_UNCOMP_MEAN    (obs) float64 ...
    TEMP_IR_SEA_WING_UNCOMP_STDDEV  (obs) float64 ...
    WAVE_DOMINANT_PERIOD            (obs) float64 ...
    WAVE_SIGNIFICANT_HEIGHT         (obs) float64 ...
    TEMP_SBE37_MEAN                 (obs) float64 ...
    TEMP_SBE37_STDDEV               (obs) float64 ...
    SAL_SBE37_MEAN                  (obs) float64 ...
    SAL_SBE37_STDDEV                (obs) float64 ...
    COND_SBE37_MEAN                 (obs) float64 ...
    COND_SBE37_STDDEV               (obs) float64 ...
    O2_CONC_SBE37_MEAN              (obs) float64 ...
    O2_CONC_SBE37_STDDEV            (obs) float64 ...
    O2_SAT_SBE37_MEAN               (obs) float64 ...
    O2_SAT_SBE37_STDDEV             (obs) float64 ...
    CHLOR_WETLABS_MEAN              (obs) float64 ...
    CHLOR_WETLABS_STDDEV            (obs) float64 ...
Attributes:
    area:                      Pacific Ocean
    cdm_data_type:             Trajectory
    cdm_trajectory_variables:  trajectory
    Conventions:               CF-1.6, ACDD-1.3, COARDS
    creator_email:             support@saildrone.com
    creator_name:              Saildrone
    creator_url:               https://saildrone.com/
    data_mode:                 realtime
    date_created:              2021-03-20T15:30:01.026538Z
    description:               Saildrone NetCDF Format
    drone_id:                  1054
    Easternmost_Easting:       -126.6228352
    featureType:               Trajectory
    geospatial_lat_max:        47.0412768
    geospatial_lat_min:        46.3617536
    geospatial_lat_units:      degrees_north
    geospatial_lon_max:        -126.6228352
    geospatial_lon_min:        -127.9267584
    geospatial_lon_units:      degrees_east
    history:                   Saildrone\n2021-03-30T18:29:56Z (local files)\...
    id:                        145302
    infoUrl:                   https://saildrone.com/
    institution:               Saildrone
    interval:                  900.0
    keywords:                  active, air, air_pressure, air_temperature, an...
    keywords_vocabulary:       GCMD Science Keywords
    license:                   2020 © SAILDRONE Inc.  All Rights Reserved.  T...
    naming_authority:          com.saildrone
    netcdf_version:            4.6.3
    nodc_template_version:

In [6]:
nc_filename = "/" + dataset_id + "_copy.nc"
data_xr.to_netcdf(path=qc_output_path + nc_filename)

In [7]:
# ---- generate QC config for each variable ---- #

# metadata for climatological data extraction
min_t = str(data_xr.time.min().dt.floor("D").dt.strftime("%Y-%m-%d").data)
max_t = str(data_xr.time.max().dt.ceil("D").dt.strftime("%Y-%m-%d").data)
min_x = float(data_xr.longitude.min().data)
min_y = float(data_xr.latitude.min().data)
max_x = float(data_xr.longitude.max().data)
max_y = float(data_xr.latitude.max().data)
bbox = [min_x, min_y, max_x, max_y]

# sus_min
# sus_max
# fail_min
# fail_max

# config for testing each variable
default_config = {
    "bbox": bbox,
    "start_time": min_t,
    "end_time": max_t,
    "tests": {
        "spike_test": {"suspect_threshold": "1", "fail_threshold": "2"},
        "gross_range_test": {
            "suspect_min": "mean - 3 * std",  # sus_min,
            "suspect_max": "mean + 3 * std",  # sus_max,
            "fail_min": "min - 3 * std",  # fail_min,
            "fail_max": "max + 3 * std",  # fail_max
        },
        # 'aggregate': {}
    },
}

# variables to apply tests to
custom_config = {
    "air_temperature": {"variable": "air"},
    "air_pressure": {"variable": "pres"},
    "relative_humidity": {"variable": "rhum"},
    "sea_water_practical_salinity": {"variable": "salinity"},
    "eastward_wind": {"variable": "uwnd"},
    "northward_wind": {"variable": "vwnd"},
    "downward_wind": {"variable": "wwnd"},
    "sea_water_temperature": {"variable": "temperature"},
}

# generate climatology configs
creator_config = {
    "datasets": [
        {
            "name": "ocean_atlas",
            "file_path": str(libdir) + "/resources/ocean_atlas.nc",
            "variables": {"o2": "o_an", "salinity": "s_an", "temperature": "t_an"},
            "3d": "depth",
        },
        {
            "name": "narr",
            "file_path": str(libdir) + "/resources/narr.nc",
            "variables": {
                "air": "air",
                "pres": "slp",
                "rhum": "rhum",
                "uwnd": "uwnd",
                "vwnd": "vwnd",
                "wwnd": "wwnd",
            },
        },
    ]
}

cc = CreatorConfig(creator_config)
qccc = QcConfigCreator(cc)

[2021-03-30 14:29:58,339] DEBUG - Loading 2 datasets...


In [8]:
# break down variable by standard name
# def not_stddev(var):
#     return var and not var.endswith('SD')

air_temp = ["air_temperature"]
pressure = ["air_pressure"]
humidity = ["relative_humidity"]
water_temp = ["sea_water_temperature"]
salt = ["sea_water_practical_salinity"]
u = ["eastward_wind"]
v = ["northward_wind"]
w = ["downward_wind"]

run_tests = air_temp + pressure + humidity + water_temp + salt + u + v + w

In [9]:
final_config = {}
for var in data_xr:
    data_arr = data_xr[var]
    # skip tests for variables not specified for testing
    if (
        "standard_name" not in data_arr.attrs
        or data_arr.attrs["standard_name"] not in run_tests
    ):
        continue
    # ignore standard deviation of variables
    if var.endswith("_STDDEV"):
        continue
    # initialize config
    config = default_config.copy()
    # set boundary values for config
    min_t = str(data_arr.time.min().dt.floor("D").dt.strftime("%Y-%m-%d").data)
    max_t = str(data_arr.time.max().dt.ceil("D").dt.strftime("%Y-%m-%d").data)
    min_x = float(data_arr.longitude.min().data)
    min_y = float(data_arr.latitude.min().data)
    max_x = float(data_arr.longitude.max().data)
    max_y = float(data_arr.latitude.max().data)
    # add time and space boundaries to config
    bbox = [min_x, min_y, max_x, max_y]
    config["bbox"] = bbox
    config["start_time"] = min_t
    config["end_time"] = max_t
    # allow custom overrides...
    # ... on a variable name basis
    if var in custom_config:
        config.update(custom_config[var])
    # ... on a standard name basis
    if data_arr.attrs["standard_name"] in custom_config:
        config.update(custom_config[data_arr.attrs["standard_name"]])
    # generate ioos_qc Config object
    L.info(config)
    qc_var = QcVariableConfig(config)
    qc_config = qccc.create_config(qc_var)
    # remove variable added by create_config
    qc_config = list(qc_config.values())[0]
    # and add it to final_config
    final_config[var] = qc_config

final_config

[2021-03-30 14:29:59,064] INFO - {'bbox': [-127.9267584, 46.3617536, -126.6228352, 47.0412768], 'start_time': '2021-01-04', 'end_time': '2021-03-15', 'tests': {'spike_test': {'suspect_threshold': '1', 'fail_threshold': '2'}, 'gross_range_test': {'suspect_min': 'mean - 3 * std', 'suspect_max': 'mean + 3 * std', 'fail_min': 'min - 3 * std', 'fail_max': 'max + 3 * std'}}, 'variable': 'uwnd'}
[2021-03-30 14:29:59,065] DEBUG - Validating schema...
[2021-03-30 14:29:59,066] DEBUG - Calculating time range start_time='1996-1-4' and end_time='1996-3-15'..
[2021-03-30 14:29:59,068] DEBUG - Subsetting by var='uwnd' bbox=[-127.9267584, 46.3617536, -126.6228352, 47.0412768] depth=0...
[2021-03-30 14:29:59,112] INFO - used bbox=[-128.42675839999998, 45.8617536, -126.1228352, 47.5412768] after padding 1 times
[2021-03-30 14:29:59,113] DEBUG - Creating config...
[2021-03-30 14:29:59,124] INFO - {'bbox': [-127.9267584, 46.3617536, -126.6228352, 47.0412768], 'start_time': '2021-01-04', 'end_time': '2021

{'UWND_MEAN': {'qartod': {'spike_test': {'suspect_threshold': 1.0,
    'fail_threshold': 2.0},
   'gross_range_test': {'suspect_span': [-1.3617822066327274,
     2.4801642776741257],
    'fail_span': [-2.096705869664451, 3.6718511262799054]}}},
 'VWND_MEAN': {'qartod': {'spike_test': {'suspect_threshold': 1.0,
    'fail_threshold': 2.0},
   'gross_range_test': {'suspect_span': [0.9948928125321121, 4.38833905848488],
    'fail_span': [-0.01270311132063795, 5.52564208760848]}}},
 'TEMP_AIR_MEAN': {'qartod': {'spike_test': {'suspect_threshold': 1.0,
    'fail_threshold': 2.0},
   'gross_range_test': {'suspect_span': [8.430926381858146, 9.222698870568163],
    'fail_span': [8.302227197112277, 9.548219750047028]}}},
 'RH_MEAN': {'qartod': {'spike_test': {'suspect_threshold': 1.0,
    'fail_threshold': 2.0},
   'gross_range_test': {'suspect_span': [81.10836690960991, 83.61780245097137],
    'fail_span': [80.32680993928021, 84.26457051844409]}}},
 'BARO_PRES_MEAN': {'qartod': {'spike_test': {

In [10]:
# c = NcQcConfig(final_config)
# nc_file = qc_output_path+nc_filename
# nc_results = c.run(nc_file)
# nc_results

In [11]:
# Run QC checks against the input netCDF file
# Some checks are computed lazily.
# Only when you iterate over the results using a
# ioos_qc.store or the collect_results function lazy
# checks actually be run.
c = Config(final_config)
xs = XarrayStream(data_xr, time="time", lat="latitude", lon="longitude")
qc_results = xs.run(c)

# Since we may want to re-use the results in multiple
# stores we want to evaluate the checks now so it isn't
# a generator. This is going to run all of the checks!
qc_results = list(qc_results)
qc_results

[<ContextResult stream_id=UWND_MEAN>,
 <ContextResult stream_id=UWND_MEAN>,
 <ContextResult stream_id=VWND_MEAN>,
 <ContextResult stream_id=VWND_MEAN>,
 <ContextResult stream_id=TEMP_AIR_MEAN>,
 <ContextResult stream_id=TEMP_AIR_MEAN>,
 <ContextResult stream_id=RH_MEAN>,
 <ContextResult stream_id=RH_MEAN>,
 <ContextResult stream_id=BARO_PRES_MEAN>,
 <ContextResult stream_id=BARO_PRES_MEAN>,
 <ContextResult stream_id=TEMP_SBE37_MEAN>,
 <ContextResult stream_id=TEMP_SBE37_MEAN>,
 <ContextResult stream_id=SAL_SBE37_MEAN>,
 <ContextResult stream_id=SAL_SBE37_MEAN>]

In [12]:
# Save the QC results to a CF netCDF file

# Import whatever DSG class you want to save the data as. You can
# figure out which type the original dataset was by loading it into pocean
from pocean.dsg import IncompleteMultidimensionalTrajectory

qc_out = os.path.join(qc_output_path, dataset_id + "_qc_out.nc")
qc_out_just_flags = os.path.join(qc_output_path, dataset_id + "_qc_out_just_flags.nc")

# Setup the CF store
store = CFNetCDFStore(qc_results)

# Store results with data and flags
if os.path.exists(qc_out):
    os.remove(qc_out)
qc_all = store.save(
    # The netCDF file to export OR append to
    qc_out,
    # The DSG class to save the results as
    IncompleteMultidimensionalTrajectory,
    # The QC config that was run
    c,
    # Should we write the data or just metadata? Defaults to false
    write_data=True,
    # Any kwargs to pass to the DSG class
    dsg_kwargs=dict(
        reduce_dims=True,  # Remove dimensions of size 1
        unlimited=False,  # Don't make the record dimension unlimited
        unique_dims=True,  # Support loading into xarray
    ),
)

# Store results only with the flags
if os.path.exists(qc_out_just_flags):
    os.remove(qc_out_just_flags)
qc_flags = store.save(
    # The netCDF file to export OR append to
    qc_out_just_flags,
    # The DSG class to save the results as
    IncompleteMultidimensionalTrajectory,
    # The QC config that was run
    c,
    # Should we write the data or just metadata? Defaults to false
    write_data=False,
    # Any kwargs to pass to the DSG class
    dsg_kwargs=dict(
        reduce_dims=True,  # Remove dimensions of size 1
        unlimited=False,  # Don't make the record dimension unlimited
        unique_dims=True,  # Support loading into xarray
    ),
)

L.info(qc_all)
L.info(qc_flags)

[2021-03-30 14:29:59,392] INFO - Adding column time from stream UWND_MEAN
[2021-03-30 14:29:59,393] INFO - Adding column lon from stream UWND_MEAN
[2021-03-30 14:29:59,394] INFO - Adding column lat from stream UWND_MEAN
[2021-03-30 14:29:59,441] INFO - Adding column time from stream UWND_MEAN
[2021-03-30 14:29:59,442] INFO - Adding column lon from stream UWND_MEAN
[2021-03-30 14:29:59,442] INFO - Adding column lat from stream UWND_MEAN
[2021-03-30 14:29:59,480] INFO - <class 'pocean.dsg.trajectory.im.IncompleteMultidimensionalTrajectory'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: CF-1.6
    date_created: 2021-03-30T18:29:00Z
    featureType: trajectory
    cdm_data_type: Trajectory
    dimensions(sizes): obs_dim(1064)
    variables(dimensions): int32 trajectory(), float64 time(obs_dim), int32 z(obs_dim), float64 lat(obs_dim), float64 lon(obs_dim), float64 UWND_MEAN(obs_dim), uint8 UWND_MEAN_qartod_spike_test(obs_dim), uint8 UWND_MEAN_qartod_gross_range_test(ob

In [13]:
from bokeh.layouts import gridplot
from bokeh.models import Legend
from bokeh.plotting import figure, output_file, output_notebook, show

output_notebook()

# Helper method to plot QC results using Bokeh
def plot_ncresults(ncdata, var_name, title, test_name):

    time = np.array(ncdata.variables["time"])
    obs = np.array(ncdata.variables[var_name])

    # Get QC Variable names
    qc_test = ncdata.get_variables_by_attributes(
        ioos_qc_test=test_name, ioos_qc_target=var_name
    )[0]
    L.info(f"Plotting {qc_test.name}")

    qc_values = qc_test[:]

    qc_pass = np.ma.masked_where(qc_values != 1, obs)
    num_pass = (qc_values == 1).sum()
    qc_suspect = np.ma.masked_where(qc_values != 3, obs)
    num_suspect = (qc_values == 3).sum()
    qc_fail = np.ma.masked_where(qc_values != 4, obs)
    num_fail = (qc_values == 4).sum()
    qc_notrun = np.ma.masked_where(qc_values != 2, obs)

    p1 = figure(
        x_axis_type="datetime",
        title="".join(
            (
                test_name,
                " : ",
                title,
                " : p/s/f=",
                str(num_pass),
                "/",
                str(num_suspect),
                "/",
                str(num_fail),
                ", ncvar=",
                qc_test.name,
            )
        ),
    )

    p1.grid.grid_line_alpha = 0.5
    p1.xaxis.axis_label = "Time"
    p1.yaxis.axis_label = "Observation Value"

    obs = p1.line(time, obs, color="#A6CEE3")
    qc_not = p1.circle(time, qc_notrun, size=2, color="gray", alpha=0.2)
    qc_pass = p1.circle(time, qc_pass, size=4, color="green", alpha=0.5)
    qc_sus = p1.circle(time, qc_suspect, size=4, color="orange", alpha=0.7)
    qc_fail = p1.circle(time, qc_fail, size=6, color="red", alpha=1.0)

    legend = Legend(
        items=[
            ("values", [obs]),
            ("notrun", [qc_not]),
            ("pass", [qc_pass]),
            ("suspect", [qc_sus]),
            ("fail", [qc_fail]),
        ],
        location="top_right",
    )

    p1.add_layout(legend, "right")

    # output_file("qc.html", title="qc example")

    show(gridplot([[p1]], plot_width=800, plot_height=400))

Loading BokehJS ...

In [14]:
# ---- plot flags for QC'd variables ---- #
from bokeh.io import show

qc_var_plots = plot_ncresults(qc_all, "UWND_MEAN", "QC", "gross_range_test")
qc_var_plots = plot_ncresults(qc_all, "VWND_MEAN", "QC", "gross_range_test")
qc_var_plots = plot_ncresults(qc_all, "TEMP_AIR_MEAN", "QC", "gross_range_test")

# show(qc_var_plots)

[2021-03-30 14:29:59,504] INFO - Plotting UWND_MEAN_qartod_gross_range_test


[2021-03-30 14:29:59,554] INFO - Plotting VWND_MEAN_qartod_gross_range_test


[2021-03-30 14:29:59,605] INFO - Plotting TEMP_AIR_MEAN_qartod_gross_range_test


In [15]:
# shutil.copy(nc_file, qc_out)

# qc_agg = data_xr
# for var in list(final_config.keys()):
#     # if var == 'UWND_MEAN':
#     #     continue
#     # if var == 'VWND_MEAN':
#     #     continue
#     for test in list(default_config['tests'].keys()):
#         qc_agg['qartod_'+var+'_'+test] = xr.Variable('time',data=nc_results[var]['qartod'][test])

# nc_qc_filename = '/'+dataset_id+'_qc.nc'
# nc_qc_file = qc_output_path+nc_qc_filename
# qc_agg.to_netcdf(path=nc_qc_file)

# print('data_xr')
# print(data_xr)
# print()
# print('qc_agg')
# print(qc_agg)